In [1]:
import snscrape.modules.twitter as sntwitter #scrapper
import pandas as pd

In [2]:
tweets_list = []
num_tweets = 1000

# Scrape tweets from Eskom account
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:Eskom_SA').get_items()): 
    if i>num_tweets:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username]) 
    
# Creating a dataframe from the tweets list above 
eskom_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

In [3]:
eskom_df.head(5)

,Datetime,Tweet Id,Text,Username
0,2023-05-17 15:30:05+00:00,1658857429692473345,Replacing all your incandescent bulbs with LED...,Eskom_SA
1,2023-05-17 13:39:42+00:00,1658829649751810050,#EskomGauteng #BuccleuchWaterfallOutage\n\nTec...,Eskom_SA
2,2023-05-17 12:45:04+00:00,1658815900022087680,Buying illegal electricity vouchers is a crime...,Eskom_SA
3,2023-05-17 10:00:30+00:00,1658774485703577601,"Eskom Acting GCEO, Calib Cassim will host the...",Eskom_SA
4,2023-05-17 08:29:02+00:00,1658751466826346496,Illegal connections kill!\nStolen electricity ...,Eskom_SA


In [4]:
shift = 31
for i in range(shift,2*shift):
    print(f"Tweet {i}:" )
    print(eskom_df.iloc[i]['Text'], "\n")

Tweet 31:
#EskomMediaStatement

Eskom notes media reports on the book by André de Ruyter https://t.co/XT2EW7BBFn 

Tweet 32:
Please note that this will be implemented until 5:00 tomorrow and the pattern as communicated yesterday will continue. 

Tweet 33:
The system operator decides on the loadshedding stage, depending on the amount of power needed to balance the grid. https://t.co/tQXytUphcX 

Tweet 34:
#LoadsheddingUpdate

Monday,15 May 2023: 

Due to the cold weather, the demand for electricity is higher than anticipated. In addition, the failure of two generating units, one of which has since returned to service, requires that Stage 6 loadshedding be implemented at 15:00 today (previously communicated 16:00). Eskom will communicate another update as soon as any significant changes occur. We appeal that non essential appliances be switched off. 

Tweet 35:
Please call your service provider if you have a power outage outside of your scheduled loadshedding time.  Eskom customers can l